# Coursera Capstone Project

#### This Notebook is made on IBM Watson Studio and is being used for the final project 'Coursera Capstone' of the Data Science IBM Professional Certificate course

In [1]:
import pandas as pd
import numpy as np
print('Libraries imported!')

Libraries imported!


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

I'll use BeautifulSoup package to scrape the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, and get the data to explore Toronto's neighborhoods.

In [3]:
from bs4 import BeautifulSoup
import requests
import csv
print('Libraries imported!')

Libraries imported!


### Scrape data using BeautifulSoup

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

csv_file = open('Toronto_zipcodes_wikipedia.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

wikitable = soup.find('table', class_='wikitable sortable')
wikitable_rows = wikitable.find_all('tr')
for tr in wikitable_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    csv_writer.writerow(row)

csv_file.close()

### Create and clean Dataframe with Pandas

In [ ]:
df = pd.read_csv('Toronto_zipcodes_wikipedia.csv')

null = df[df['Borough'] == str('Not assigned')].index
df.drop(null, inplace=True)

df = df.groupby(by=['Postcode', 'Borough']).agg(lambda x:', '.join(x))
df.reset_index(level=['Postcode', 'Borough'], inplace=True)

df['Neighbourhood'] = df['Neighbourhood'].replace('\n','', regex=True)

print('Dataframe shape:', df.shape)
df.head()

Dataframe shape: (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Get zipcodes

In [ ]:
!pip install geocoder
import geocoder
print('Geocoder imported!')

Geocoder imported!


### Retrieve list of zipcodes and add them to the dataframe

In [ ]:
code_list = df['Postcode'].tolist()
lat_list=[]
lng_list=[]

for postcode in code_list:
    latlng = None
    while (latlng is None):
        g = geocoder.arcgis('{}, Toronto, ON'.format(postcode))
        latlng = g.latlng
    lat_list.append(g.latlng[0])
    lng_list.append(g.latlng[1])

df['Latitude'] = lat_list
df['Longitude'] = lng_list

df.head()

## Explore and Cluster Toronto Neighbourhoods

In [ ]:
from pandas.io.json import json_normalize
import json
from geopy.geocoders import Nominatim
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
!pip install folium
import folium
print('Libraries imported!')

### Visualize Toronto and its Neighborhoods

In [ ]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='ca_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto coordinates are: {},{}'.format(latitude, longitude))

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, neighbourhood, borough in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['Borough']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_opacity=0.3, parse_html=False).add_to(map_toronto)
map_toronto

#### SEE MAP ON GITHUB: https://github.com/toom90034/Coursera_Capstone/blob/master/Toronto_Boroughs.png

### Foursquare Credentials

In [ ]:
# @hidden cell
CLIENT_ID = 'YHUWGBGBBLPF5RGHJQV3JLAZC1QFCSTM5RT0EOIQ5EEHYYF0'
CLIENT_SECRET = 'YOAWQ5F5MFWR51Y1F00UAS2L3N0APZSQYRWTS2IOMUJ24UJR'
VERSION = '20180604'
print('Foursquare Credentials saved!')

### We will focus our exploration on Downtown Toronto

In [ ]:
downtown_df = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()

### Get the coordinates of Downtown Toronto

In [ ]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

### Visualize Downtown Toronto Neighborhoods

In [ ]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### SEE MAP ON GITHUB: https://github.com/toom90034/Coursera_Capstone/blob/master/Toronto_Downtown_Neighborhoods.png

### Top 100 venues in Downtown Toronto

In [ ]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
results

### Structure the results in a new Dataframe

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

### Explore the neighborhoods of Downtown

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
downtown_venues = getNearbyVenues(names=downtown_df['Neighbourhood'],
                                   latitudes=downtown_df['Latitude'],
                                   longitudes=downtown_df['Longitude']
                                  )

### Analyze each neighborhood

In [ ]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.head()

In [ ]:
downtown_onehot.shape

### Group by neighborhood and the mean of frequency of occurence for each category

In [ ]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

### Check the new size

In [ ]:
downtown_grouped.shape

### Take a look at the Top5 most common venue for each neighborhood

In [ ]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Save the results in a new dataframe 

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Now Let's cluster the neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

### Visualize the results

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### SEE MAP ON GITHUB: https://github.com/toom90034/Coursera_Capstone/blob/master/Toronto_Downtown_Venues_Clusters.png

### Examine the clusters

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

In [ ]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

## We can clearly see the clustered neighborhoods and also notice that coffee shops are everywhere! 